# Finding the bulk modulus of model aluminium

If you haven't already done so, please run the first two cells in the "GettingGoing" notebook to set up your Google Drive and clone this repository.

## Install lammmps software


In [ ]:
# Install lammps
!pip install lammps[mpi]
# Install package to assist in parsing lammps log files
!pip install lammps-logfile

## Mount your Google drive
...and change to the correct directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/cdt_training/MaterialsModellingPrimer/02-BulkModulus

## Import some useful modules 

In [1]:
import numpy as np
from scipy import optimize
import os
import shutil
import subprocess
from contextlib import chdir
import lammps_logfile
import matplotlib.pyplot as plt
%matplotlib inline

## Select potential

In [ ]:
pair_styles = ['eam/alloy', 'eam/fs', 'eam/alloy', 'eam/alloy', 'eam/alloy', 'eam/alloy']
potential_files = ['Pot1.set', 'Pot2.eam.fs', 'Pot3.eam.alloy', 'Pot4.eam.alloy', 'Pot5.eam.alloy', 'Pot6.eam.fs']
potential_index = 1

potential_file = potential_files[potential_index]
pair_style = pair_styles[potential_index]

print('You have selected potential: ', potential_file)

## Build and run the simulations
The below code uses a loop over the values of the variable `scale` to create and run multiple simulations with different values for the lattice parameter. 

In [ ]:
# Define a function to replace text in base input file
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

a = [4.0248454, 4.0452598, 4.0319604, 4.05, 4.050005, 4.0500102]    # Equilibrium lattice parameter
scale = [0.980, 0.985, 0.990, 0.995, 
         1.000, 1.005, 1.010, 1.015, 1.020]     # Factors by which to scale lattice constant
ncells = [5]                                    # Number of unit cells in each direction

# Loop over values of unit cell size and number of unit cells in simulation
# Create a folder and lammps input files for each variant
for i in range(len(scale)):
    for j in range(len(ncells)):
        # Create  a folder for simulation and copy in the empirical potential file
        path = './Simulations/' + 'potential_' + str(potential_index+1) +  '/' + 'Scale_'+ str(scale[i]) + '/' + 'Cells_' + str(ncells[j]) + '/'
        if not os.path.exists(path):
            os.makedirs(path, mode=0o777)
        shutil.copy2('../Potentials/' + potential_file, path) 
        
        # Define values to replace in base input file
        replacements = {
            'ALATT':str(scale[i]*a[potential_index]),
      		'NCELLS':str(ncells[j]),
            'POTENTIAL':potential_file,
            'PAIRSTYLE':pair_style
        }
        
        # Create a lammps file and use base file with string replacements to create unique variant
        inputLammpsFile = 'in_base.lmps'
        outputLammpsFile = path + 'in.lmps'        
        finLammps = open(inputLammpsFile, 'r').read()
        foutLammps = open(outputLammpsFile, 'w')
        out = replace_all(finLammps, replacements)
        foutLammps.write(out)
        foutLammps.close()

        lammps_executable = 'lmp'
        lammps_command = '-in in.lmps'

        with chdir(path):
            os.system(lammps_executable + ' ' + lammps_command)

## Gather results
Loop over all the simulations and gather the values of quantities of interest into a single file.

In [4]:
output_path = './Simulations/' + 'potential_' + str(potential_index+1) + '/'

with chdir(output_path):

    output_file = 'results.txt'
    colwidth = 20
    fo = open(output_file,'w')
    fo.write('#Strain'.ljust(colwidth) + 'Lattice Param (A)'.ljust(colwidth) + 'Volume (A^3)'.ljust(colwidth) + 'Energy (eV)'.ljust(colwidth) + 'Pressure (GPa)'.ljust(colwidth) + '\n')

    for i in range(len(scale)):
        for j in range(len(ncells)):
            # Create  a folder for simulation and copy in the empirical potential file
            path = 'Scale_'+ str(scale[i]) + '/' + 'Cells_' + str(ncells[j]) + '/'
            
            log = lammps_logfile.File(path + 'log.lammps')

            step = log.get("Step")
            pe = log.get("TotEng")
            length = log.get("Lx")
            pressure = log.get("Pxx")   

            fo.write(
                str(scale[i]).ljust(colwidth) + 
                str(length[-1]/float(ncells[j])).ljust(colwidth) + 
                str(length[-1]**3).ljust(colwidth) + 
                str(pe[-1]).ljust(colwidth) + 
                str(pressure[-1]/10000.0).ljust(colwidth) + '\n'
                )
    fo.close()

## Load in the simulation results
Read the results of our simulations into a numpy array. Numpy makes this incredibly easy: one line is enough.

In [5]:
data = np.loadtxt(output_path + 'results.txt', skiprows=1)

## Plot the data
The strain is in the first column of the array and the enrgy in the fourth column. We can easily plot a graph to examine the form of the data.

In [ ]:
fig,axes = plt.subplots(1,1, figsize=(4,4))
axes.plot(data[:,0],data[:,3], 'o-')
axes.set_xlabel('Scaling')
axes.set_ylabel('Energy (eV)')

## An equation of state
A common way of extracting the bulk modulus from data of this sort is to use the Birch-Murnaghan equation of state:
$$
    E(V) =  E_0 + \frac{9V_0B_0}{16}\left\lbrace \left[   \left( \frac{V_0}{V}  \right)^{\frac{2}{3}}  -1  \right]^3 B_0^\prime + \left[   \left( \frac{V_0}{V}  \right)^{\frac{2}{3}}  -1  \right]^2 \left[  6 -4 \left( \frac{V_0}{V}  \right)^{\frac{2}{3}}  \right]  \right\rbrace  \; .
$$
This relates the energy of our crystal to the volume. $E_0$ and $V_0$ are respectively the energy and volume of the crystal at the equilibrium volume (scaling = 1.0). $B_0$ is then the bulk modulus and $B_0'$ is its first derivative with respect to pressure.

We can encode this equation of state in a python function. Here I am setting the values of $V_0$ and $E_0$ with the values from the results file. Since $V_0$ and $V$ mostly appear in the quotient $V_0/V$, I am defining this in the third line of the function. The rest of the lines just encode the equation of state. It could all be done on a single line, but I've built up the result in stages to reduce the chances of errors and to make any debugging easier (maths isn't easy to read in code form). 

In [7]:
def BM(V,B0,B0p):
    V0 = data[int((np.shape(data)[0]-1)/2),2]
    E0 = data[int((np.shape(data)[0]-1)/2),3]
    v = V0/V
    E = np.power(np.power(v,2/3)-1,3)*B0p
    E = E + np.power(np.power(v,2/3)-1,2) * (6 - 4*np.power(v,2/3))
    E = E * (9*V0*B0/16)
    E = E + E0
    return E

## Finding the best fit
Python has lots of useful tools in the `scipy` library. Here we are importing the `optimise` module and using it to find the values of $B_0$ and $B_0'$ that give the best fit to the data for the equation of state. These values will be stored in the variable `params`. Again, all it takes is a single line of code:

In [8]:
params, params_covariance = optimize.curve_fit(BM, data[:,2], data[:,3])#,p0=[8200, 2])

We can check the result by plotting it against the data:

In [ ]:
fig,axes = plt.subplots(1,1, figsize=(4,4))
V_vals = np.linspace(7750,8850,100)
axes.plot(V_vals,BM(V_vals,params[0],params[1]), '-', label='Best fit')
axes.plot(data[:,2],data[:,3], 'o', label='Simulation')
axes.set_xlabel(r'Volume ($\AA^3$)')
axes.set_ylabel('Energy (eV)')
axes.legend()

The fit looks good.

## Assessing the result 
We now have our estimate of the bulk modulus $B_0$ from the simulation. To compare it to the experimental value we need to get it into the right units. Currently it is in electron volts per cubic angstrom. We'd like it in gigapascal. That's an easy conversion to make:

In [ ]:
eV = 1.602e-19
A = 1e-10
GPa = 1e9
print(f'Our estimate of the bulk modulus is {params[0]*eV/A**3/GPa:0.1f} GPa')


A quick estimate for Al from Google is $B_0=$ 75 GPa. This is a pretty reasonable match.
